__собираем данные__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/house/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_house.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/house/avito/avito_2022-09-15_11-54_raw_sevastopol.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-42_raw_bahchisaray.xlsx',
 'data/raw/house/avito/avito_2022-09-15_14-47_raw_sudak.xlsx']

In [7]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

2816


In [8]:
# data_avito

In [9]:
from lib.avito import AvitoDataCleanerRealtyHouse

data_avito = AvitoDataCleanerRealtyHouse().transform( data_avito )

print(len(data_avito))

2816


In [10]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
1248,2513432254,дом 100 м² на участке 4 сот.,8900000,,"СТ Милосердие, 163","Дом 96 м2 по цене квартиры! \nДом новой постройки для круглогодичного проживания, район Фиолент Снт Милоседие. \n\nДом построен качественно: ленточный фундамент 1 м, монолитная плита, колоны и сейсмо-пояса, заполнение ракушечником в камень наружные стены и в",26,2022-09-15 11:53:29.976,sevastopol,100.00,False,False,True,4.00,8.90,2-4,70-150
36,2534686669,дом 112 м² на участке 5 сот.,5800000,,ул. Ревякина,"В продаже жилой дом по ул. Ревякина. Общей площадью 113 кв.м., три жилые комнаты, кухня, санузел, крыша шифер . Котёл угольный . \nУчасток 5 соток .\nДо центра города 5 мин.",1,2022-09-15 11:53:29.976,sevastopol,112.00,False,False,False,5.00,5.80,4-8,70-150
1951,2495875417,дом 486 м² на участке 4 сот.,200000000,,"ул. Военных Строителей, 68А","- Фундамент — монолитный ленточный;\n- Каркас — монолитный ж/б каркас с заполнением проёмов и наружных стен ракушечником и газоблоком;\n- Кровля — плоская, эксплуатируемая (мембрана Пвх Технониколь 8 мм, утеплитель экструдированный пенополистерол Carbon Prof",41,2022-09-15 11:53:29.976,sevastopol,486.00,False,False,False,4.00,200.00,2-4,300+


In [12]:
data_avito['place'].unique()

array(['sevastopol', 'bahchisaray', 'sudak'], dtype=object)

In [13]:
assert len(data_avito)>0

In [14]:
data_avito.info(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2816 entries, 0 to 2815
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   avito_id             2816 non-null   int64         
 1   title                2816 non-null   object        
 2   price                2816 non-null   int64         
 3   obj_name             2816 non-null   object        
 4   adr                  2816 non-null   object        
 5   description          2816 non-null   object        
 6   avito_page           2816 non-null   int64         
 7   ts                   2816 non-null   datetime64[ns]
 8   place                2816 non-null   object        
 9   house_area           2816 non-null   float64       
 10  is_part              2816 non-null   bool          
 11  is_townhouse         2816 non-null   bool          
 12  is_SNT               2816 non-null   bool          
 13  land_area            2816 non-nul

In [15]:
data_avito.isnull().any()

avito_id               False
title                  False
price                  False
obj_name               False
adr                    False
description            False
avito_page             False
ts                     False
place                  False
house_area             False
is_part                False
is_townhouse           False
is_SNT                 False
land_area              False
priceM                 False
land_size_category      True
house_size_category     True
dtype: bool

In [16]:
data_avito[ data_avito['land_size_category'].isnull() ]

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,house_area,is_part,is_townhouse,is_SNT,land_area,priceM,land_size_category,house_size_category
2283,2337799298,,11900000,,Восточная ул.,"Предлагаю 1/2 дома общей площадью 120 кв. М. В центре города в отличном районе для проживания на земельном участке 1.6 сотки. Вся инфраструктура рядом, транспортные развязки в шаговой доступности. При этом дом расположен в тихом месте, с каждого этажа дома",47,2022-09-15 11:53:29.976,sevastopol,0.00,True,False,False,0.00,11.90,NaN,NaN
2614,2034310938,,5000000,,"с. Танковое, ул. Ходырева","Собственник. Срочно продам дом 74,3 кв м на земельном участке 814 м кв. Санузел и туалет в доме, 2 спальни. 1965 года постройки, камень бут, очень тёплый зимой и прохладный летом. Газ, приборы учета на все коммуникации. В дом есть 2 входа. Также на участке",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,False,0.00,5.00,NaN,NaN
2624,2391096190,,3000000,,"с. Растущее, Садовая ул., 8","Продам Дом 75 КВ. М, 10 Соток Земли Ижс Продам дом в живописном районе Крыма, в селе Растущее 10 соток земли, 75 кв. М, 4 комнаты, коммуникации: свет, газ, вода, скважина, отопление Агв. Дом чистый, уютный, заходи живи. Участок ровный, сад, весь обработанн",4,2022-09-15 14:42:27.254,bahchisaray,0.00,False,False,True,0.00,3.00,NaN,NaN
2733,2418227584,,5500000,,"Платановый квартал, 21",Недострой (Ижс).\nЗемля 5.2 сотки (собственность).\nВсе вопросы по телефону.,1,2022-09-15 14:47:45.056,sudak,0.00,False,False,False,0.00,5.50,NaN,NaN


---

In [17]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

2816


### обновляем таблицу адресов

In [20]:
# data = data.sample(10)

In [18]:
# import pandas as pd
# loc_file_path = 'data/location_yandex.pkl'# таблица адресов
# df=pd.read_pickle(loc_file_path)
# # df

# df['latitude_'] =  df['latitude']
# df['latitude']  =  df['longitude']
# df['longitude'] = df['latitude_']
# df = df.drop(columns=['latitude_',])

# df.to_pickle(loc_file_path)

In [19]:
# from lib.locator import LocationUpdater
# from lib.locator import AddressTransformerSev

# loc_file_path = 'data/location.pkl'# таблица адресов

# loc = LocationUpdater(
#         address_transformer=AddressTransformerSev(),
#     ).transform(
#         adr=data.sample(3)['adr'],
#         loc=pd.read_pickle(loc_file_path.pkl'),
#         show_pbar=True,
#     )

----

In [22]:
# from lib.locator import AddressTransformer

# data['adr'] = AddressTransformer().transform( data['adr'] )

0       Севастополь, Балаклавский район , садовое товарищество Успех, 21
1                                       садовое товарищество Тополёк, 10
2                                                 ул. Николая Музыки, 51
3                                         СТ Яблоня, 2-я ул. Сарандинаки
4                                                      ул. Будённого, 55
                                      ...                               
2811                                                       квартал Асрет
2812                                                       ул. Ашик-Умер
2813                                      с. Солнечная Долина, ул. Авдет
2814                                           с. Богатовка, Зелёная ул.
2815                                                   ул. Пограничников
Name: adr, Length: 2816, dtype: object

In [ ]:
# data.loc[ data['place'] == 'bahchisaray', 'adr' ]
# data.loc[ data['place'] == 'sudak', 'adr' ]
# data.loc[ data['place'] == 'sevastopol', 'adr' ]

----

In [ ]:
from lib.locator import LocationUpdater
from lib.locator import GeocoderYandex
from yandex_keys import keys


loc_file_path = 'data/location.pkl'# таблица адресов

loc = LocationUpdater(
        locator=GeocoderYandex(keys[0]),
    ).transform(
        adr=data['adr'],
        loc=pd.read_pickle(loc_file_path),
        show_pbar=True,
    )

In [ ]:
loc

In [ ]:
loc.sample(3)

In [ ]:
assert len(loc)>0
loc.to_pickle(loc_file_path)

### дополняем данные геометкой

In [ ]:
data = data.merge(loc[['adr','latitude','longitude',]],on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isnull() ] ) )

In [ ]:
assert len(data)>0
data.to_pickle(result_file_path)

In [ ]:
data.info()

In [ ]:
# for f in raw_data_files:
#     ts = dtm.strptime( re.sub(r'.*/avito_','',f), '%Y-%m-%d_%H-%M_raw.xlsx')
#     df = pd.read_excel(f)
#     df['ts'] = ts
#     df.to_excel(f,index=False)
#     print(ts)

In [ ]:
# class AvitoDataCleanerRealtyLand:
    
#     def transform(self,data):
#         df = data.copy()
        
#         symb_lat = 'yexapocEXAPOCTHKBM'
#         symb_rus = 'уехаросЕХАРОСТНКВМ'
#         l2r = str.maketrans(symb_lat,symb_rus) 
        
#         df['title'] = df['title'].str.extract( r'«(.*)»',expand=False).apply(lambda s: s.translate(l2r) )
        
#         area = df['title'].str.extract( r'(\d+,?\d*)\s*(сот|га)',expand=True)
#         area.columns=['area','area_unit']
#         df = pd.concat([df,area],axis=1)
                       
#         df['is_IJS'] = df['title'].str.lower().str.match(r'.*ижс.*')
#         df['obj_name'] = df['obj_name'].fillna('')
#         df['price'] = df['price'].astype(int)
#         df['priceM'] = df['price']/1e6
        
#         df['area'] = df['area'].fillna('0.').str.replace(',','.').astype(float)
        
#         df.loc[ df['area_unit']=='га', 'area' ] = df.query('area_unit=="га"')['area']*100.
#         df = df.drop(columns=['area_unit'])
        
#         return df

In [ ]:
# data_avito['title'].str.extract( r'(\d+,?\d*)\s*(сот|га)',expand=True) #, columns=['a','b'])
        

In [ ]:
# # import re
# # s='Участок 9,7 сот. (ИЖС)'
# s='Участок 7,8 сот. (ИЖС)'
# s='Участок 7,8 га. (ИЖС)'

# s='Участок 7,8 м. (ИЖС)'

# # s='Участок 5 сот. (ИЖС)'

# re.search( r'(\d+,?\d*)\s*(сот|га)',s) #.group(1)

# # # len(s)

----

In [ ]:
## собираем данные циан.ру

In [ ]:
# import re
# from os import listdir

# raw_data_files = sorted([
#         cian_raw_data_path+'/'+f 
#         for f in listdir(cian_raw_data_path) 
#         if re.match(r'.+\.pkl$',f)
#         #if re.match(r'.+\.xlsx$',f)
    
#     ])
# raw_data_files

In [ ]:
# data_cian = pd.concat([
#         # pd.read_excel(f)
#         pd.read_pickle(f)
#          .dropna()
#          .drop_duplicates(['LinkArea','Description',])
#          .rename(columns={'page':'cian_page'})
#          .reset_index(drop=True)  
#         for f in raw_data_files
#     ]).reset_index(drop=True)

In [ ]:
# data_cian.info(2)

In [ ]:
# swap_title = data_cian['OfferSubtitle'].str.match(r'.*\d+ м², \d+/\d+ эт.*') 

# data_cian = pd.concat([
#         data_cian[ ~swap_title ],
#         data_cian[ swap_title ]
#             .rename(columns={'OfferTitle':'OfferSubtitle','OfferSubtitle':'OfferTitle'})
#     ]).reset_index(drop=True)


In [ ]:
# data_cian

In [ ]:
# from lib.cian import CianDataCleaner

# data_cian = CianDataCleaner().transform( data_cian )

# print(len(data_cian))

In [ ]:
# assert len(data_cian)>0

In [ ]:
# data_cian.info(2)

In [ ]:
# data_cian
# data_cian.query('nrooms==0')
# data_cian[ data_cian['title'].str.len()<1 ]
# data_cian[ data_cian['obj_name'].str.len()>0 ][['title','obj_name',]] # .to_csv('tmp/cian_title.csv',sep='\t')|

----

In [ ]:
# data = pd.concat([data_avito,data_cian]).reset_index(drop=True) 
# print(len(data))
# assert len(data)>0